## Inteligencia Artificial
## - Laboratorio 5 -
Autores:
- Mark Albrand 
- Jimena Hernández

### Task 1 - Graph-Search

In [1]:
import matplotlib.pylab as plt
import numpy as np

### Task 1.1 - Discretización de la imagen

In [2]:
from PIL import Image
import numpy as np
def convertir_a_discreto(imagen_path, tamano_bloque):
    # Cargar la imagen
    imagen = Image.open(imagen_path)
    imagen_np = np.array(imagen)
    
    # Dimensiones de la imagen
    alto, ancho, _ = imagen_np.shape
    
    nuevo_alto = (alto // tamano_bloque) + 1
    nuevo_ancho = (ancho // tamano_bloque) + 1

    print(f'Alto: {alto}, Ancho: {ancho}')
    print(f'Nuevo alto: {nuevo_alto}, Nuevo ancho: {nuevo_ancho}')

    # Redimensionar la imagen
    imagen_salida = np.zeros((nuevo_alto, nuevo_ancho, 3), dtype=np.uint8)
    
    for y in range(0, alto, tamano_bloque):
        for x in range(0, ancho, tamano_bloque):
            new_y = y // tamano_bloque
            new_x = x // tamano_bloque
            # Extraer el bloque
            bloque = imagen_np[y:y+tamano_bloque, x:x+tamano_bloque]
            
            # Calcular el color promedio del bloque
            color_promedio = bloque.mean(axis=(0,1)).astype(int)
            #convertir los colores difuminados al color mas predominante
            #verde
            if color_promedio[1] > color_promedio[0] and color_promedio[1] > color_promedio[2]:
                color_promedio[1] = 255
                color_promedio[0] = 0
                color_promedio[2] = 0 
            #rojo
            elif color_promedio[0] > color_promedio[1] and color_promedio[0] > color_promedio[2]:
                color_promedio[0] = 255
                color_promedio[1] = 0
                color_promedio[2] = 0

            #grises
            elif color_promedio[0] == color_promedio[1] and color_promedio[0] == color_promedio[2] and color_promedio[0]< 255:
                color_promedio[0] = 0
                color_promedio[1] = 0
                color_promedio[2] = 0
            
            # Asignar el color promedio al bloque correspondiente en la imagen de salida
            imagen_salida[new_y, new_x] = color_promedio
    
    imagen_discreta = Image.fromarray(imagen_salida)
    return imagen_discreta

ruta_imagen = 'images/1.bmp' 
tamano_bloque = 10 #bloques de 14x14
imagen_discreta = convertir_a_discreto(ruta_imagen, tamano_bloque)
imagen_discreta.show() # Mostrar la imagen


Alto: 582, Ancho: 582
Nuevo alto: 59, Nuevo ancho: 59


Las áreas blancas representan caminos libres
Las áreas negras representan paredes sobre las cuales no se puede pasar
Las áreas verdes representan la meta (goalTest positivos) (pueden ser varios)
La área roja representa el punto de inicio (solo podrá haber uno)

In [3]:
print(type(imagen_discreta))

<class 'PIL.Image.Image'>


In [4]:
# Pasar la imagen a un arreglo de numpy
imagen_np = np.array(imagen_discreta)
print(imagen_np.shape)

height, width, channels = imagen_np.shape

(59, 59, 3)


In [5]:
print(imagen_np)

[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[  0   0   0]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]


### Task 1.2 - Framework de Problemas

Puede entenderse como un framework con los siguientes
elementos
- ○ Estado inicial (s_start) CHEQUE
- ○ Función de acción actions(s) → {a 1, a 2,...,a n} CHEQUE
- ○ Función de costo de paso cost(s,a,s’) → R CHEQUE
-    Estado actual, acción y estado siguiente -> Cuánto cuesta hacer el moviemiento
- ○ Función de resultados results(s,a) → s’
-    Se usa después de actions, dice a que estado se llega usando lo que dio actions
- ○ Función de goalTest(s) → {True, False}
    Se pasa de cada config nueva (estado actual), si es la meta o no

● Además, algunas veces es útil agregar
- ○ Función de costo de ruta pathCost(s 1, s 2,...,s n) → R
■ s i = results(s i-1,a i )

In [6]:
from typing import TypedDict

class legend_dict(TypedDict):
    """
    Define el tipo de dato para el diccionario de leyenda
    Los numpy arrays son de 3 elementos, representando los colores RGB
    """
    wall: np.array
    path: np.array
    start: np.array
    end: np.array


class SolveMaze:
    maze: np.array
    starting_point: tuple[int, int]  # (y, x)

    legend: legend_dict = {
        "wall": np.array([0, 0, 0]),  # black
        "path": np.array([255, 255, 255]),  # white
        "start": np.array([255, 0, 0]),  # red
        "end": np.array([0, 255, 0])  # green
    }

    def __init__(self, maze: np.array):
        if maze.ndim != 3:
            raise ValueError("Maze debe ser un arreglo tridimensional")

        self.maze = maze
        self.starting_point = self._findStartinPoint()

    
    def _findStartinPoint(self) -> tuple[int, int]:
        for y in range(0, height, 1):
            for x in range(0, width, 1):
                if np.array_equal(self.maze[y, x], self.legend["start"]):  # Buscar el punto de inicio, definido por el color en legend
                    return (y, x)
        return None
    
    def __str__(self) -> str:
        return f"SolveMaze(maze={self.maze.shape}, starting_point={self.starting_point})"
    
    def actions(self, current_pos: tuple[int, int]) -> list[tuple[int, int]]:
        """
        Dada una posición, regresa las posiciones a las que se puede mover
        """
        y, x = current_pos
        possible_actions = []
        if y > 0 and np.array_equal(self.maze[y-1, x], self.legend["path"]):
            # Si no estamos en el borde superior y la celda de arriba es un camino
            # Arriba
            possible_actions.append((y-1, x))
        if y < height-1 and np.array_equal(self.maze[y+1, x], self.legend["path"]):
            # Si no estamos en el borde inferior y la celda de abajo es un camino
            # Abajo
            possible_actions.append((y+1, x))
        if x > 0 and np.array_equal(self.maze[y, x-1], self.legend["path"]):
            # Si no estamos en el borde izquierdo y la celda de la izquierda es un camino
            # Izquierda
            possible_actions.append((y, x-1))
        if x < width-1 and np.array_equal(self.maze[y, x+1], self.legend["path"]):
            # Si no estamos en el borde derecho y la celda de la derecha es un camino
            # Derecha
            possible_actions.append((y, x+1))
        
        return possible_actions
    
    def pathCost(self, next_pos: tuple[int, int]) -> int:
        """
        Dada una posición y una acción, regresa el costo de llegar a esa posición
        """
        actions_next = self.actions(next_pos)
        costo = 1

        # Cantidad de acciones posibles
        match len(actions_next):
            case 0:
                costo += 999
            case 1:
                costo += 3
            case 2:
                costo += 2
            case 3:
                costo += 1
            case 4:
                pass
            case _:
                pass

        
        # De las acciones posibles, si una es el punto final, reducir el costo
        for action in actions_next:
            if np.array_equal(action, legend_dict["end"]):
                costo -= 15
                break


        return costo
        

    def goalTest(self, current_pos: tuple[int, int]) -> bool:
        """
        Dada una posición, regresa si es el punto final
        """
        return np.array_equal(self.maze[current_pos], self.legend["end"])


In [9]:
path = SolveMaze(imagen_np)
print(path)
print(path.actions(path.starting_point))

SolveMaze(maze=(59, 59, 3), starting_point=(49, 48))
[(48, 48), (49, 47)]
